In [3]:
import os
import random
import json
import time

In [4]:
import cloudscraper
from bs4 import BeautifulSoup
import geopandas as gpd
import pandas as pd

In [5]:
from shapely.geometry import Point
from tqdm.notebook import tqdm

## Get State Data

In [6]:
file_path = "data/cb_2018_us_state_500k/cb_2018_us_state_500k.shp"
states_df = gpd.read_file(file_path)

In [7]:
states_df = states_df[["STUSPS", "NAME", "geometry"]]

## Get Data

In [8]:
scraper = cloudscraper.create_scraper()

In [9]:
store_url = "https://find.shell.com/us/fuel/10005366-229-broadway-ave-n"

In [10]:
def get_store_point(store_url: str) -> dict:
    r = scraper.get(store_url)
    soup = BeautifulSoup(r.text, "html.parser")
    store_dict = json.loads(soup.find("script", {"type": "application/ld+json"}).text)
    point = Point(
        float(store_dict["location"]["geo"]["longitude"]),
        float(store_dict["location"]["geo"]["latitude"]),
    )
    return point

In [11]:
def get_store_urls(url: str) -> list:
    r = scraper.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    props = json.loads(
        soup.find("div", {"data-react-class": "pages/StationListDirectoryPage"}).attrs[
            "data-react-props"
        ]
    )["stationListProps"]
    urls = [
        "https://find.shell.com" + location["link"] for location in props["locations"]
    ]
    return urls

In [12]:
def get_urls(url: str) -> list:
    r = scraper.get(url)
    soup = BeautifulSoup(r.text, "html.parser")
    props = json.loads(
        soup.find("div", {"data-react-class": "pages/GeographicDirectoryPage"}).attrs[
            "data-react-props"
        ]
    )["geographicListProps"]
    urls = [
        "https://find.shell.com" + location["link"] for location in props["locations"]
    ]
    return urls

In [13]:
state_urls = get_urls("https://find.shell.com/us/fuel/locations/")

# Parse all locations in a state
for state_url in tqdm(state_urls, desc="Parsing States"):
    state_store_list = []
    state_name = state_url.split("/")[-1].replace("-", " ").lower()

    # If File exists continue
    if os.path.isfile(f"data/states/{state_name}.gpkg"):
        continue

    city_urls = get_urls(state_url)

    if not city_urls:
        print(state_code)
        continue

    for city_url in tqdm(city_urls, desc=f"Parsing Locations in {state_name}"):
        store_urls = get_store_urls(city_url)
        if not store_urls:
            print("Error getting city url", city_url)
            continue

        for store_url in store_urls:
            try:
                point = get_store_point(store_url)
                store_dict = {"geometry": point, "NAME": state_name, "url": store_url}
            except Exception as e:
                print("Error getting store url", store_url, str(e))
                continue
            state_store_list.append(store_dict)
            time.sleep(random.uniform(0.01, 0.5))

    if state_store_list:
        store_gdf = gpd.GeoDataFrame(state_store_list, crs=4326)
        store_gdf.to_file(f"data/states/{state_name}.gpkg")

Parsing States:   0%|          | 0/50 [00:00<?, ?it/s]

Parsing Locations in virginia:   0%|          | 0/170 [00:00<?, ?it/s]

Parsing Locations in washington:   0%|          | 0/149 [00:00<?, ?it/s]

Parsing Locations in west virginia:   0%|          | 0/13 [00:00<?, ?it/s]

Parsing Locations in wisconsin:   0%|          | 0/127 [00:00<?, ?it/s]

In [14]:
store_gdfs = []
states_path = "data/states"
for file in os.listdir("data/states"):
    constructed_path = os.path.join(states_path, file)
    if constructed_path.endswith(".gpkg"):
        gdf = gpd.read_file(constructed_path)
        store_gdfs.append(gdf)

In [15]:
gdf = pd.concat(store_gdfs, ignore_index=True)

In [16]:
gdf.head()

,NAME,url,geometry
0,alabama,https://find.shell.com/us/fuel/10010285-501-ki...,POINT (-85.25077 31.56329)
1,alabama,https://find.shell.com/us/fuel/10001657-1091-1...,POINT (-86.81049 33.23352)
2,alabama,https://find.shell.com/us/fuel/10004112-718-1s...,POINT (-86.81512 33.25161)
3,alabama,https://find.shell.com/us/fuel/10010287-9737-h...,POINT (-86.82431 33.22074)
4,alabama,https://find.shell.com/us/fuel/10013271-687-hi...,POINT (-86.19496 34.28147)


In [17]:
gdf = gdf.to_crs(9311)
gdf.to_file(f"data/stores.gpkg")

## Get Population Data

In [18]:
state_populations = pd.read_excel(
    "data/NST-EST2024-POP.xlsx", sheet_name=None, engine="openpyxl"
)

In [19]:
state_populations_df = state_populations["NST-EST2024-POP"][
    [
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)",
        "Unnamed: 5",
    ]
]
state_populations_df = state_populations_df.rename(
    columns={
        "table with row headers in column A and column headers in rows 3 through 4. (leading dots indicate sub-parts)": "NAME",
        "Unnamed: 5": "POPULATION",
    }
)
state_populations_df["NAME"] = state_populations_df["NAME"].str[1:]

In [20]:
states_with_population_df = states_df.merge(state_populations_df, on="NAME", how="left")
states_with_population_df = states_with_population_df[
    ["STUSPS", "NAME", "POPULATION", "geometry"]
]

In [21]:
states_with_population_df = states_with_population_df.dropna()

In [28]:
states_with_population_df["NAME"] = states_with_population_df["NAME"].str.lower()

## Merge Data

In [29]:
count_df = pd.DataFrame(gdf.groupby("NAME").size(), columns=["COUNT"]).reset_index()

In [30]:
count_gdf = states_with_population_df.merge(count_df, on="NAME", how="left")
count_gdf = count_gdf.fillna(0)

In [31]:
count_gdf["per_100k"] = (count_gdf["COUNT"] / (count_gdf["POPULATION"] / 100000)).round(
    decimals=2
)
count_gdf["per_1m"] = (
    count_gdf["COUNT"] / (count_gdf["POPULATION"] / 1_000_000)
).round(decimals=1)

In [34]:
count_gdf = count_gdf.to_crs(9311)

In [35]:
count_gdf.to_file("data/Shell_Gas_Stations_Per_State.gpkg")

In [36]:
count_gdf[["COUNT", "per_100k", "per_1m", "NAME"]].sort_values(
    "per_100k", ascending=False
)

,COUNT,per_100k,per_1m,NAME
0,259.0,8.80,88.0,mississippi
23,609.0,8.52,85.2,tennessee
5,329.0,7.17,71.7,louisiana
17,359.0,7.01,70.1,alabama
30,45.0,6.94,69.4,vermont
51,231.0,6.34,63.4,connecticut
48,194.0,6.32,63.2,arkansas
42,69.0,6.25,62.5,rhode island
18,691.0,6.25,62.5,georgia
43,279.0,6.13,61.3,kentucky
